<a href="https://colab.research.google.com/github/zesameri/pytorch-tutorial/blob/main/logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
# prepare data

bc = datasets.load_breast_cancer()
x, y = bc.data, bc.target
_, num_features = x.shape
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2, random_state=1234)

# scale
# this fits the features so the mean is 0 and unit variance
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
Y_train = torch.from_numpy(Y_train.astype(np.float32))
Y_test = torch.from_numpy(Y_test.astype(np.float32))

Y_train = Y_train.view(Y_train.shape[0], 1) # now it is a column vector
Y_test = Y_test.view(Y_test.shape[0], 1) 

In [4]:
# f = wq + b, sigmoid at the end

class LogisticRegression(nn.Module):
  def __init__(self, num_input_features):
    super(LogisticRegression, self).__init__()
    self.linear = nn.Linear(num_input_features, 1)

  def forward(self, x):
    y_hat = torch.sigmoid(self.linear(x))
    return y_hat

model = LogisticRegression(num_features)

In [7]:
# loss and optimizer
learning_rate= 0.01
criterion = nn.BCELoss() #Binary Cross Entropy Loss
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [13]:
# training
num_epochs = 1000
for epoch in range(num_epochs):
  # forward pass & loss
  y_hat = model(X_train)
  loss = criterion(y_hat, Y_train)

  # backward pass
  loss.backward()

  # updates
  optimizer.step()
  optimizer.zero_grad()

  if (epoch + 1) % 100 == 0:
    print(f'epoch: {epoch + 1}, loss = {loss.item():.4f}')

epoch: 100, loss = 0.0830
epoch: 200, loss = 0.0805
epoch: 300, loss = 0.0783
epoch: 400, loss = 0.0764
epoch: 500, loss = 0.0746
epoch: 600, loss = 0.0730
epoch: 700, loss = 0.0715
epoch: 800, loss = 0.0701
epoch: 900, loss = 0.0689
epoch: 1000, loss = 0.0677


In [14]:
# evaluation
with torch.no_grad():
  y_hat = model(X_test)
  y_hat_classes = y_hat.round()
  accuracy = y_hat_classes.eq(Y_test).sum() / float(Y_test.shape[0])
  print(f'accuracy = {accuracy:.4f}')

accuracy = 0.9561
